In [651]:
import os
import numpy as np
import pandas as pd

In [652]:

file_path_1 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/fundamental_all.csv' # Fundamental data
df1 = pd.read_csv(file_path_1)
df1['datadate'] = pd.to_datetime(df1['datadate'])
df1 = df1.drop(columns=['fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr','costat'])

df1 = df1.sort_values(by=['tic', 'datadate'])

df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17745 entries, 1236 to 15067
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   gvkey     17745 non-null  int64         
 1   datadate  17745 non-null  datetime64[ns]
 2   tic       17745 non-null  object        
 3   actq      13578 non-null  float64       
 4   ancq      13192 non-null  float64       
 5   epsfxq    16905 non-null  float64       
 6   lctq      13579 non-null  float64       
 7   ltq       15674 non-null  float64       
 8   revtq     17110 non-null  float64       
 9   ugiq      496 non-null    float64       
 10  capxy     13254 non-null  float64       
 11  prccq     17377 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 1.8+ MB


In [653]:
df1.head()

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,ugiq,capxy,prccq
1236,1690,1979-12-31,AAPL,23.052,1.584,0.06,11.689,12.262,19.540,NaN,NaN,NaN
1237,1690,1980-03-31,AAPL,28.367,1.986,0.06,14.244,14.632,23.550,NaN,NaN,NaN
1238,1690,1980-06-30,AAPL,37.930,3.777,0.06,20.976,21.926,32.569,NaN,NaN,NaN
1239,1690,1980-09-30,AAPL,54.106,11.244,0.07,37.780,39.401,41.467,NaN,NaN,NaN
1240,1690,1980-12-31,AAPL,145.720,16.402,0.14,28.866,31.117,67.621,NaN,NaN,34.124969


In [654]:
file_path_2 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/ratios_all.csv' # Financial ratios data
df2 = pd.read_csv(file_path_2)
df2['qdate'] = pd.to_datetime(df2['qdate'])
df2['public_date'] = pd.to_datetime(df2['public_date'])

# Sort by qdate and public_date to ensure the most recent public_date comes last
df2 = df2.sort_values(by=['qdate', 'public_date'])
# Drop duplicates to keep only the first public_date for each qdate
df2 = df2.drop_duplicates(subset=['qdate','TICKER'], keep='first')

columns_to_keep = [
    'gvkey', 'qdate', 'roa',
    'roe','pe_inc', 'divyield',
    'pcf', 'roce', 'fcf_ocf',
    'debt_ebitda', 'gpm', 'opmad',
    'evm', 'intcov_ratio', 'at_turn',
    'cfm', 'npm', 'PEG_trailing',
    'de_ratio', 'curr_ratio', 'ptb',
]

df2 = df2[columns_to_keep]

# convert divyield to float
df2['divyield'] = df2['divyield'].replace('%', '', regex=True).astype(float)

df2 = df2.sort_values(by=['qdate'])

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15533 entries, 0 to 26692
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   gvkey         15533 non-null  int64         
 1   qdate         15520 non-null  datetime64[ns]
 2   roa           15472 non-null  float64       
 3   roe           15267 non-null  float64       
 4   pe_inc        14787 non-null  float64       
 5   divyield      11974 non-null  float64       
 6   pcf           15467 non-null  float64       
 7   roce          15353 non-null  float64       
 8   fcf_ocf       14206 non-null  float64       
 9   debt_ebitda   15355 non-null  float64       
 10  gpm           15512 non-null  float64       
 11  opmad         15512 non-null  float64       
 12  evm           15437 non-null  float64       
 13  intcov_ratio  14432 non-null  float64       
 14  at_turn       15475 non-null  float64       
 15  cfm           15197 non-null  float6

In [655]:
df = pd.merge(df1, df2, left_on=['datadate', 'gvkey'], right_on=['qdate', 'gvkey'], how='inner')

columns_to_keep = [
    'gvkey', 'datadate', 'tic',
    'actq', 'ancq', 'epsfxq',
    'lctq', 'ltq', 'revtq',
    'capxy', 'prccq', 'roa',
    'roe','pe_inc', 'divyield',
    'pcf', 'roce', 'fcf_ocf',
    'debt_ebitda', 'gpm', 'opmad',
    'evm', 'intcov_ratio', 'at_turn',
    'cfm', 'npm', 'PEG_trailing',
    'de_ratio', 'curr_ratio', 'ptb',
]

df = df[columns_to_keep]

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15523 entries, 0 to 15522
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   gvkey         15523 non-null  int64         
 1   datadate      15523 non-null  datetime64[ns]
 2   tic           15523 non-null  object        
 3   actq          12739 non-null  float64       
 4   ancq          12361 non-null  float64       
 5   epsfxq        15404 non-null  float64       
 6   lctq          12742 non-null  float64       
 7   ltq           14742 non-null  float64       
 8   revtq         15076 non-null  float64       
 9   capxy         12419 non-null  float64       
 10  prccq         15502 non-null  float64       
 11  roa           15469 non-null  float64       
 12  roe           15264 non-null  float64       
 13  pe_inc        14789 non-null  float64       
 14  divyield      11974 non-null  float64       
 15  pcf           15464 non-null  float6

In [656]:
df = df.sort_values(by=['tic','datadate'])

df.head()

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,prccq,roa,roe,pe_inc,divyield,pcf,roce,fcf_ocf,debt_ebitda,gpm,opmad,evm,intcov_ratio,at_turn,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb
0,1690,1980-12-31,AAPL,145.720,16.402,0.14,28.866,31.117,67.621,NaN,34.124969,0.851,0.868,80.303,NaN,-74.325,1.240,NaN,0.027,0.433,0.202,13.535,56.192,2.206,0.112,0.100,NaN,0.556,2.612,11.083
1,1690,1981-03-31,AAPL,173.152,17.587,0.16,39.428,43.693,78.765,NaN,24.499985,0.606,0.468,77.035,NaN,-85.319,1.587,NaN,0.020,0.440,0.194,17.753,45.675,1.917,0.112,0.104,NaN,0.420,3.234,12.188
2,1690,1981-06-30,AAPL,201.262,21.085,0.21,49.914,58.268,90.719,NaN,25.999985,0.539,0.388,34.698,NaN,-56.890,2.136,NaN,0.028,0.463,0.206,18.791,54.142,1.740,0.112,0.115,NaN,0.368,3.681,6.507
3,1690,1981-09-30,AAPL,227.104,27.734,0.19,70.280,77.451,97.678,NaN,15.249996,0.467,0.376,26.607,NaN,-44.727,0.583,NaN,0.182,0.487,0.198,11.468,50.879,2.091,0.143,0.118,NaN,0.437,3.231,5.659
4,1690,1981-12-31,AAPL,249.160,36.026,0.24,83.952,92.894,133.553,NaN,22.124985,0.409,0.286,22.812,NaN,-180.620,0.429,NaN,0.031,0.503,0.186,14.316,50.879,1.682,0.140,0.114,NaN,0.400,3.492,5.086


In [657]:
file_path_3 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/s&p500index.csv' # s&p500
df3 = pd.read_csv(file_path_3)
df3['caldt'] = pd.to_datetime(df3['caldt'])  # Adjust column name if needed

df = df.sort_values(by=['datadate'])


#merge (fill empty cells with nearest s&p500 value)
df = pd.merge_asof(df, df3, left_on='datadate', right_on='caldt', direction='nearest')
df = df.drop(columns=['caldt'])

# calculate relative returns
#df['relative_quarterly_return'] = (df['prccq'] - df['spindx'])
#df = df.drop(columns=['prccq', 'spindx'])

#df = df.dropna(subset=['prccq'])

df = df.sort_values(by=['tic','datadate']).reset_index()
df.tail()

,index,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,prccq,roa,roe,pe_inc,divyield,pcf,roce,fcf_ocf,debt_ebitda,gpm,opmad,evm,intcov_ratio,at_turn,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb,spindx
15518,15149,13721,2022-09-30,ZTS,6551.0,7123.0,1.13,2880.0,9012.0,2003.0,415.0,148.29,0.241,0.421,35.192,0.843,38.833,0.252,0.685,2.027,0.742,0.358,28.597,10.901,0.581,0.315,0.258,2.637,1.993,2.506,14.605,3585.62
15519,15190,13721,2022-12-31,ZTS,7506.0,7419.0,0.99,3167.0,10522.0,2041.0,586.0,146.55,0.236,0.449,37.194,0.898,40.474,0.246,0.694,2.389,0.736,0.364,22.351,12.149,0.561,0.319,0.262,2.791,2.389,2.370,17.019,3839.50
15520,15288,13721,2023-03-31,ZTS,6269.0,7485.0,1.19,1915.0,9263.0,2034.0,223.0,166.44,0.241,0.430,36.797,0.920,35.004,0.249,0.678,2.097,0.737,0.358,23.885,11.502,0.580,0.312,0.255,3.695,2.094,2.450,16.287,4109.31
15521,15385,13721,2023-06-30,ZTS,6183.0,7566.0,1.45,1769.0,9128.0,2181.0,389.0,172.21,0.249,0.467,40.023,0.787,43.869,0.259,0.643,2.033,0.743,0.365,23.067,11.678,0.589,0.325,0.268,3.257,2.085,2.724,18.478,4450.38
15522,15517,13721,2023-09-30,ZTS,6222.0,7884.0,1.29,1608.0,9032.0,2152.0,534.0,173.98,0.255,0.484,35.909,0.849,36.919,0.265,0.679,1.986,0.744,0.367,23.285,11.605,0.595,0.328,0.271,2.875,2.040,3.095,15.485,4288.05


In [658]:
df = df.sort_values(by=['tic','datadate'])

# convert to pct
df['prccq'] = df['prccq'].pct_change()
df['spindx'] = df['spindx'].pct_change()

df['relative_quarterly_return'] = (df['prccq'] - df['spindx'])
df['target'] = df['relative_quarterly_return'].shift(-1)

df = df.drop(columns=['prccq', 'spindx'])

df.tail()

,index,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,roa,roe,pe_inc,divyield,pcf,roce,fcf_ocf,debt_ebitda,gpm,opmad,evm,intcov_ratio,at_turn,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb,relative_quarterly_return,target
15518,15149,13721,2022-09-30,ZTS,6551.0,7123.0,1.13,2880.0,9012.0,2003.0,415.0,0.241,0.421,35.192,0.843,38.833,0.252,0.685,2.027,0.742,0.358,28.597,10.901,0.581,0.315,0.258,2.637,1.993,2.506,14.605,-0.084526,-0.082539
15519,15190,13721,2022-12-31,ZTS,7506.0,7419.0,0.99,3167.0,10522.0,2041.0,586.0,0.236,0.449,37.194,0.898,40.474,0.246,0.694,2.389,0.736,0.364,22.351,12.149,0.561,0.319,0.262,2.791,2.389,2.370,17.019,-0.082539,0.065449
15520,15288,13721,2023-03-31,ZTS,6269.0,7485.0,1.19,1915.0,9263.0,2034.0,223.0,0.241,0.430,36.797,0.920,35.004,0.249,0.678,2.097,0.737,0.358,23.885,11.502,0.580,0.312,0.255,3.695,2.094,2.450,16.287,0.065449,-0.048332
15521,15385,13721,2023-06-30,ZTS,6183.0,7566.0,1.45,1769.0,9128.0,2181.0,389.0,0.249,0.467,40.023,0.787,43.869,0.259,0.643,2.033,0.743,0.365,23.067,11.678,0.589,0.325,0.268,3.257,2.085,2.724,18.478,-0.048332,0.046754
15522,15517,13721,2023-09-30,ZTS,6222.0,7884.0,1.29,1608.0,9032.0,2152.0,534.0,0.255,0.484,35.909,0.849,36.919,0.265,0.679,1.986,0.744,0.367,23.285,11.605,0.595,0.328,0.271,2.875,2.040,3.095,15.485,0.046754,NaN


In [659]:
# GETTING RID OF ERRORS CAUSED BY PCT CHANGE

# Create a list to track the indices to drop
indices_to_drop = []

# Initialize the previous ticker as None
previous_ticker = None

# Iterate through the DataFrame, skipping the first row
for idx, row in df.iterrows():
    current_ticker = row['tic']
    
    # If the current ticker is different from the previous one, mark the current row for deletion
    if previous_ticker is not None and current_ticker != previous_ticker:
        indices_to_drop.append(idx)  # Append the index of the first occurrence of the new ticker
        indices_to_drop.append(idx-1)  # Append the index of the first occurrence of the new ticker
    
    # Update the previous ticker
    previous_ticker = current_ticker

# Drop the rows with the indices in the list
df = df.drop(indices_to_drop)

# Reset the index of the DataFrame after dropping rows
df = df.reset_index(drop=True)

df = df.drop(columns = ['index'])

df.tail(50)

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,roa,roe,pe_inc,divyield,pcf,roce,fcf_ocf,debt_ebitda,gpm,opmad,evm,intcov_ratio,at_turn,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb,relative_quarterly_return,target
15289,4503,2021-12-31,XOM,59154.0,279769.0,2.08,56643.0,163240.0,81305.0,12076.0,0.129,0.127,14.549,4.490,6.898,0.107,0.749,1.220,0.195,0.087,7.185,15.029,0.824,0.155,0.083,NaN,0.968,1.044,1.759,-0.066181,0.399198
15290,4503,2022-03-31,XOM,77255.0,277516.0,1.28,72059.0,178245.0,87734.0,3911.0,0.151,0.143,15.920,3.670,7.537,0.146,0.747,1.072,0.200,0.105,6.603,36.592,0.898,0.147,0.084,NaN,1.039,0.968,2.143,0.399198,0.201380
15291,4503,2022-06-30,XOM,93163.0,274611.0,4.21,80110.0,183266.0,111265.0,7748.0,0.194,0.212,10.458,3.680,6.228,0.219,0.771,0.771,0.219,0.136,5.354,58.737,1.008,0.166,0.111,NaN,1.027,1.053,2.011,0.201380,0.072272
15292,4503,2022-09-30,XOM,99289.0,270863.0,4.68,74057.0,177109.0,106512.0,12624.0,0.226,0.275,9.089,3.270,6.010,0.273,0.781,0.611,0.233,0.155,4.801,74.063,1.081,0.187,0.134,NaN,1.001,1.163,2.211,0.072272,0.192510
15293,4503,2022-12-31,XOM,97631.0,271436.0,3.09,69045.0,166594.0,93164.0,18407.0,0.243,0.274,8.289,3.310,5.894,0.296,0.760,0.544,0.244,0.172,5.770,41.949,1.126,0.193,0.140,NaN,0.854,1.414,2.077,0.192510,-0.076075
15294,4503,2023-03-31,XOM,97224.0,272147.0,2.79,66666.0,162957.0,83644.0,5412.0,0.245,0.304,6.918,3.560,5.273,0.316,0.746,0.504,0.255,0.183,4.994,93.824,1.069,0.211,0.156,NaN,0.911,1.336,1.862,-0.076075,-0.104976
15295,4503,2023-06-30,XOM,91760.0,271488.0,1.94,61815.0,156251.0,80795.0,10771.0,0.213,0.245,8.895,3.270,6.568,0.261,0.684,0.558,0.246,0.168,5.912,74.357,0.990,0.199,0.142,NaN,0.851,1.421,2.000,-0.104976,0.132793
15296,13721,2013-03-31,ZTS,2942.0,3200.0,0.28,1287.0,5418.0,1098.0,22.0,0.188,0.106,34.409,0.813,5.220,0.215,0.722,1.405,0.677,0.226,6.283,21.717,0.723,0.138,0.105,NaN,1.069,2.657,3.029,-0.811283,-0.108942
15297,13721,2013-06-30,ZTS,3082.0,3179.0,0.26,1321.0,5438.0,1119.0,80.0,0.187,0.129,34.702,0.892,26.027,0.220,0.722,1.857,0.673,0.225,8.912,14.271,0.721,0.126,0.095,NaN,1.545,2.561,13.154,-0.108942,-0.033787
15298,13721,2013-09-30,ZTS,3083.0,3176.0,0.26,1308.0,5393.0,1111.0,135.0,0.190,0.143,39.936,0.835,22.475,0.227,0.740,2.466,0.672,0.228,12.668,11.043,0.724,0.129,0.086,NaN,2.896,2.376,14.006,-0.033787,-0.048750


In [660]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15339 entries, 0 to 15338
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   gvkey                      15339 non-null  int64         
 1   datadate                   15339 non-null  datetime64[ns]
 2   tic                        15339 non-null  object        
 3   actq                       12620 non-null  float64       
 4   ancq                       12244 non-null  float64       
 5   epsfxq                     15225 non-null  float64       
 6   lctq                       12626 non-null  float64       
 7   ltq                        14606 non-null  float64       
 8   revtq                      14898 non-null  float64       
 9   capxy                      12298 non-null  float64       
 10  roa                        15297 non-null  float64       
 11  roe                        15099 non-null  float64       
 12  pe_i

In [661]:
df = df.sort_values(by=['relative_quarterly_return'])

df.head(50)

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,roa,roe,pe_inc,divyield,pcf,roce,fcf_ocf,debt_ebitda,gpm,opmad,evm,intcov_ratio,at_turn,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb,relative_quarterly_return,target
2704,2176,2010-03-31,BRK.B,NaN,NaN,1.51,NaN,204396.000,31018.000,1170.000,0.083,0.101,12.533,NaN,4.708,0.128,0.674,1.849,0.193,0.166,8.838,9.543,0.395,0.137,0.110,NaN,1.318,NaN,0.464,-1.047903,0.099181
6065,22260,1990-06-30,HCA,94.653,132.917,0.05,50.753,174.786,62.773,4.316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.510,-1.042564,-0.006817
13968,10920,1999-12-31,UPS,11138.000,11905.000,0.56,4198.000,10569.000,7446.000,1587.000,0.256,0.074,70.942,1.240,2.688,0.323,0.286,0.473,0.190,0.147,16.767,16.081,1.349,0.075,0.033,NaN,0.847,2.653,0.403,-1.008573,-0.107089
10001,7343,1984-08-31,MU,42.676,78.378,0.36,29.844,31.747,37.435,65.133,0.574,0.635,35.362,NaN,15.706,0.645,-1.168,0.080,0.608,0.428,12.795,21.318,1.161,0.401,0.332,NaN,0.355,1.430,5.172,-1.008170,-0.085401
12020,8898,1983-09-30,RJF,NaN,NaN,0.36,49.971,54.154,18.166,NaN,NaN,NaN,6.636,NaN,NaN,NaN,NaN,1.125,0.177,0.166,4.614,4.996,NaN,0.082,0.072,NaN,1.911,NaN,1.539,-0.974499,-0.152226
3604,20779,1990-04-30,CSCO,72.168,4.115,0.26,11.448,11.700,17.851,3.079,0.881,1.160,32.716,NaN,NaN,2.201,NaN,0.016,0.640,0.291,4.446,NaN,1.739,0.156,0.186,NaN,0.410,3.311,5.041,-0.961988,-0.002948
13813,10903,1984-12-31,UNH,9.784,5.903,-0.09,7.605,8.515,12.580,1.204,0.413,0.296,43.125,NaN,211.693,0.492,-3.394,0.000,0.356,0.166,9.722,15.268,2.118,0.129,0.089,NaN,0.245,4.421,8.437,-0.941643,0.916109
4683,138743,2000-09-30,EQIX,346.256,354.452,-0.70,90.273,288.005,3.933,272.400,-0.182,-6.533,-32.037,NaN,-4.202,-0.387,NaN,-2.830,-2.268,-15.491,-7.427,-2.892,0.010,-15.957,-17.396,NaN,1.081,4.435,0.812,-0.935173,-0.426131
8594,160225,2014-03-31,MA,11342.000,3460.000,0.73,5994.000,8224.000,2172.000,25.000,0.390,0.439,28.601,0.576,22.424,0.634,0.958,0.009,0.586,0.555,16.824,328.429,0.629,0.405,0.374,NaN,0.906,1.920,12.853,-0.923562,-0.063407
11891,23812,1991-06-30,REGN,111.338,3.823,-0.02,2.034,5.890,1.742,0.127,-0.183,-0.502,-40.401,NaN,-26.637,-0.442,NaN,-0.028,-0.678,-0.756,-13.429,-512.500,0.123,-0.291,-0.369,NaN,0.130,23.656,0.482,-0.917658,0.299541


In [662]:
# Save DataFrame to CSV file
df.to_csv('processed_data.csv', index=False)  # Set index=False to exclude the index column